### 📌 01. LLM 에 도구 바인딩
- LLM 단독 → 단순 텍스트 생성
- **Tools** 바인딩 → 검색, 계산, DB 질의 등 기능 확장
- `load_tools()`로 불러와서 Agent에 결합
- Agent는 질문을 보고 적절한 Tool을 선택하여 실행


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [2]:
# pip install duckduckgo-search

from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import initialize_agent, load_tools

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# DuckDuckGo 툴 직접 로드
ddg_search = DuckDuckGoSearchRun()
tools = load_tools(["llm-math"], llm=llm) + [ddg_search]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

agent.run("서울의 현재 날씨를 검색하고, 256의 제곱근을 계산해줘")

C:\Users\james\AppData\Local\Temp\ipykernel_3904\2506427008.py:13: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
C:\Users\james\AppData\Local\Temp\ipykernel_3904\2506427008.py:15: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` in



> Entering new AgentExecutor chain...
I should use DuckDuckGo search to find the current weather in Seoul and then use the calculator to calculate the square root of 256.
Action: duckduckgo_search
Action Input: "current weather in Seoul"
Observation: 16시간 전 — Current weather condition is Partly cloudy with real-time temperature (27°C), humidity 89%, wind 6.5km/h, pressure (1013mb), UV (0), visibility (10km) in ... 2일 전 — Get Seoul, 11, KR current weather report with temperature, feels like, wind ... Seoul, 11, KR Current Weather. Seoul, 11, KR. Updated 6 minutes ago. 31. °C. 7시간 전 — TodayHourly14 DaysPastClimate. Currently: 84 °F . Partly sunny. (Weather station: Seoul / Kimp'O International Airport, South Korea). See more current weather. 2025. 8. 4. — This is a weather conditions table that shows the location, weather , temperature , precipitation, wind, air pressure, etc. Time, Weather , Temp.(℃) ... 2025. 8. 17. — Check the weather forecast for Seoul. Current temperature: 25°C . 

'The current weather in Seoul is partly cloudy with a temperature of 27°C, and the square root of 256 is 16.'

### 📌 02. Agent, AgentExecutor 생성
- `Agent` = 실행 전략(도구 선택 방식)
- `AgentExecutor` = Agent + Tool 실행 관리자
- `invoke({"input": ...})` 로 실행
- `initialize_agent` vs `AgentExecutor`
  - 전자는 빠른 초기화
  - 후자는 세밀한 제어 가능


In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_openai_functions_agent, load_tools

# 1) LLM 준비
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 2) 사용할 툴 
tools = load_tools(["llm-math"], llm=llm)

# 3) 프롬프트: 반드시 agent_scratchpad 자리 포함!
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# 4) 함수형 에이전트 생성
agent = create_openai_functions_agent(llm, tools, prompt)

# 5) 실행기
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 6) 호출
executor.invoke({"input": "256의 제곱근은 뭐야? 그리고 3^4는?"})



> Entering new AgentExecutor chain...

Invoking: `Calculator` with `sqrt(256)`


Answer: 16.0
Invoking: `Calculator` with `3^4`


Answer: 81256의 제곱근은 16이고, 3의 4제곱은 81입니다.

> Finished chain.


{'input': '256의 제곱근은 뭐야? 그리고 3^4는?',
 'output': '256의 제곱근은 16이고, 3의 4제곱은 81입니다.'}

### 📌 03. 중간단계 스트리밍
- `streaming=True` 설정 → 토큰 단위 스트리밍 출력
- `AgentStreamParser` 활용 → **Tool 선택, Thought, Observation** 단계별 추적 가능
- 디버깅/모니터링에 유용


In [5]:
# Agent 의 중간단계 스트리밍 (AgentStreamParser)
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

streaming_llm = ChatOpenAI(model="gpt-3.5-turbo", streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

agent = create_openai_functions_agent(streaming_llm, tools, prompt)
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

executor.invoke({"input": "3의 제곱근은?"})



> Entering new AgentExecutor chain...

Invoking: `Calculator` with `sqrt(3)`


Answer: 1.73205080756887723의 제곱근은 1.7320508075688772 입니다.3의 제곱근은 1.7320508075688772 입니다.

> Finished chain.


{'input': '3의 제곱근은?', 'output': '3의 제곱근은 1.7320508075688772 입니다.'}

### 📌 04. Agent 메모리 추가
- 기본 Agent는 상태 없음 → 멀티턴 대화 불가
- `ConversationBufferMemory` 추가 → 이전 대화 저장
- 다양한 메모리 적용 가능:
  - BufferMemory
  - SummaryMemory
  - TokenBufferMemory


In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_openai_functions_agent, load_tools
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 툴은 예시로 llm-math만
tools = load_tools(["llm-math"], llm=llm)

# 프롬프트에 chat_history와 agent_scratchpad 둘 다 반드시 포함!
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder("chat_history"),          # ← 메모리 주입 지점
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),      # ← 에이전트 중간 단계 기록
])

agent = create_openai_functions_agent(llm, tools, prompt)

# 기존 메모리와 키 이름 맞추기
memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True
)

executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

executor.invoke({"input": "내 이름은 DataWizardd야"})
print(executor.invoke({"input": "내 이름이 뭐였지?"})["output"])



> Entering new AgentExecutor chain...
안녕하세요, DataWizardd님! 무엇을 도와드릴까요?

> Finished chain.


> Entering new AgentExecutor chain...
당연히 DataWizardd님이세요! 어떻게 도와드릴까요?

> Finished chain.
당연히 DataWizardd님이세요! 어떻게 도와드릴까요?


### 📌 05. 다양한 LLM 기반 Agent
- LangChain은 여러 LLM Provider 지원
- **OpenAI, Anthropic(Claude), Google(Gemini), TogetherAI, Ollama** 등
- 동일한 Agent 구조에 다른 LLM 바인딩만 교체 → 호환성 높음
- 👉 상황/비용/성능 요구에 따라 LLM 교체 가능


In [7]:
# 다양한 LLM 을 활용한 에이전트 생성

# OpenAI GPT
gpt_llm = ChatOpenAI(model="gpt-4o-mini")

# Anthropic Claude (예시: langchain-anthropic 필요)
# from langchain_anthropic import ChatAnthropic
# claude_llm = ChatAnthropic(model="claude-3-opus-20240229")

# Google Gemini (예시: langchain-google-genai 필요)
# from langchain_google_genai import ChatGoogleGenerativeAI
# gemini_llm = ChatGoogleGenerativeAI(model="gemini-pro")

# TogetherAI
# from langchain_community.chat_models import ChatTogether
# together_llm = ChatTogether(model="together/gpt-neoxt")

# Ollama (로컬 실행)
# from langchain_community.chat_models import ChatOllama
# ollama_llm = ChatOllama(model="llama2")

print("LLM 다양하게 교체 가능")

LLM 다양하게 교체 가능


### 📌 06. iter() & Human-in-the-loop
- `executor.iter()` → Agent 실행 과정을 단계별 반환
- 각 단계: Thought → Action → Observation
- Human-in-the-loop:
  - 실행 중간에 사람이介入해 결과 수정/결정 가능
  - 보안/품질 요구가 높은 워크플로우에 적합


In [8]:
# iter() 함수로 단계별 출력과 Human-in-the-loop

for step in executor.iter({"input": "대한민국의 수도는 어디야?"}):
    print("단계 출력:", step)
    # 사람이 중간에 intervention 가능 (예: Tool 결과 수정)



> Entering new None chain...
대한민국의 수도는 서울입니다.대한민국의 수도는 서울입니다.

> Finished chain.
단계 출력: {'output': '대한민국의 수도는 서울입니다.', 'messages': [AIMessage(content='대한민국의 수도는 서울입니다.', additional_kwargs={}, response_metadata={})]}


C:\Users\james\anaconda3\envs\my_new_env\lib\site-packages\langchain\chains\base.py:487: UserWarning: 'ConversationBufferMemory' got multiple output keys: dict_keys(['output', 'messages']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  self.memory.save_context(inputs, outputs)


# ✅ 최종 정리
- **Binding Tools** → Agent 기능 확장
- **Agent & Executor** → 전략 + 실행 관리
- **Streaming & Parser** → 단계별 스트리밍 모니터링
- **Memory 추가** → 멀티턴 대화 지원
- **다양한 LLM 교체** → GPT, Claude, Gemini, Together, Ollama
- **iter()** → 단계별 실행 확인 + Human-in-the-loop介入

👉 Agent는 LLM을 **도구 활용 가능한 지능적 실행자**로 확장하는 핵심 모듈
